In [1]:
import sqlite3
import logging

#c = conn.cursor()
def create_db():
    """Create a sqlite3 database in which we can store matches from the dota2 api."""
    conn = sqlite3.connect('dota2data')
    conn.execute(
    "CREATE TABLE if not exists matches\
    (\
    match_id INTEGER PRIMARY KEY,\
    start_time INTEGER NOT NULL,\
    lobby_type INTEGER NOT NULL,\
    radiant_hero_1 INTEGER NOT NULL,\
    radiant_hero_2 INTEGER NOT NULL,\
    radiant_hero_3 INTEGER NOT NULL,\
    radiant_hero_4 INTEGER NOT NULL,\
    radiant_hero_5 INTEGER NOT NULL,\
    dire_hero_1 INTEGER NOT NULL,\
    dire_hero_2 INTEGER NOT NULL,\
    dire_hero_3 INTEGER NOT NULL,\
    dire_hero_4 INTEGER NOT NULL,\
    dire_hero_5 INTEGER NOT NULL,\
    game_mode INTEGER,\
    has_leaver INTEGER,\
    duration INTEGER,\
    radiant_win INTEGER\
    )")
    conn.commit()
    conn.close()

api_key = 'CB3762621E46727A6F9FC9B72F6A9DE5'
database = 'dota2data'

logging.info("Creating database because it does not exist.")
create_db()

In [2]:
import dota2api
import requests
import time
import json
import sqlite3
import logging
from threading import Thread 

def match_history():
    count_history = 1
    while(count_history>=0):
        do_match_history()
        time.sleep(60)
        #print('Inside match_history')
        count_history = count_history - 1
        
def do_match_history(lobby_types=[0, 2, 5, 6, 7], skill=3, tournament_games_only=False,start_match_id_local = 4015357757):
    #start_match_id = 4015357757
    #print('inside do_match_history')
    matches = 0
    count = 100*1000
    while(count>0):
    #while(True)
        tries = 0
        while tries < 3:
            tries+=1
            try:
                result = api.get_match_history(skill = skill, min_players = 10, tournament_games_only="1" if tournament_games_only else "0", 
                                               matches_requested=100, match_id=4021439876,)
                break
            except(requests.exceptions.ConnectionError,  dota2api.src.exceptions.APIError, dota2api.src.exceptions.APITimeoutError):
                logging.warning("Api or timeout error in MatchHistory on try {}.".format(tries))
                time.sleep(30)
                continue
        if tries==3:
            return
        if result["results_remaining"] == 0:
            break
        start_at_match_id = result["matches"][-1]["match_id"] - 1
        for match in result["matches"]:
            if len(match["players"]) == 10 and match["lobby_type"] in lobby_types:
                add_match_to_db(match)
                matches += 1
        count = count - 1
    print("Added", matches, "matches.")

    
def add_match_to_db(match):
    #print('Inside add_match_to_db')
    radiant_heros = list()
    dire_heros = list()
    assert(len(match["players"])==10)
    for p in match["players"]:
        slot = p["player_slot"]
        if slot< 128:
            radiant_heros.append(p["hero_id"])
        else:
            dire_heros.append(p["hero_id"])
    assert(len(radiant_heros) == 5)
    assert(len(dire_heros) == 5)
    # Skip matches where a player didnt pick a hero
    if 0 in radiant_heros or 0 in dire_heros:
        logging.info("Skipping {} because of 0 hero.".format(match["match_id"]))
        return
    connection = sqlite3.connect(database, timeout=10)
    connection.execute("INSERT OR IGNORE INTO matches values (?,?,?,?,?,?,?,?,?,?,?,?,?,null,null,null,null)",
                       (match["match_id"], match["start_time"], match["lobby_type"]) + tuple(radiant_heros) + tuple(dire_heros))
    connection.commit()
    connection.close()

api_key = 'CB3762621E46727A6F9FC9B72F6A9DE5'
database = 'dota2data'

api = dota2api.Initialise(api_key,raw_mode=True)
#match_history_thread = Thread(target=match_history)
#match_history_thread.daemon = True
#match_details_thread = Thread(target=match_details)
#match_details_thread.daemon = True
print("Starting MatchHistory and MatchDetails threads.")
#match_history_thread.start()
#match_details_thread.start()
#match_history() Uncomment this line
# This is needed (instead of joining the threads) to respond to crl-c
#while True:
#    time.sleep(1)


# def match_details():
#     count_matches = 10
#     while(count_matches>=0):
#         do_match_details()
#         time.sleep(60)
#         print('Inside match_details')
#         count_matches = count_matches - 1
        
    
# def do_match_details():
#     print('Inside do_match_details')
#     connection = sqlite3.connect(database, timeout=10)
#     matches = connection.execute("SELECT match_id FROM matches WHERE radiant_win IS NULL").fetchall()
#     connection.close()
#     for (match_id,) in matches:
#         try:
#             result = api.get_match_details(match_id=match_id)
#         except (requests.exceptions.ConnectionError,  dota2api.src.exceptions.APIError, dota2api.src.exceptions.APITimeoutError):
#             logging.warning("Api or timeout error in MatchDetails on matchid {}.".format(match_id))
#             time.sleep(30)
#             continue
#         print("Added details for {}.".format(result["match_id"]))
#         update_match_in_db(result)
#         time.sleep(1)
    

# def update_match_in_db():
#     print('Inside update_match_in_db')
#     has_leavers = False
#     for p in match["players"]:
#         if p["leaver_status"] in [2,3,4,5,6]: 
#             has_leavers = True
#         connection = sqlite3.connect(database, timeout = 10)
#         connection.execute("UPDATE OR IGNORE matches SET game_mode=?, has_leaver=?, duration=?, radiant_win=? WHERE match_id=?", (match["game_mode"], has_leavers, match["duration"], match["radiant_win"], match["match_id"]))
#         connection.commit()
#         connection.close()
        


    

Starting MatchHistory and MatchDetails threads.


In [5]:

def match_details():
    count_matches = 5
    while(count_matches>=0):
        do_match_details()
        time.sleep(60)
        print('Inside match_details')
        count_matches = count_matches - 1
        
    
def do_match_details():
    print('Inside do_match_details')
    connection = sqlite3.connect(database, timeout=10)
    matches = connection.execute("SELECT match_id FROM matches WHERE radiant_win IS NULL").fetchall()
    connection.close()
    for (match_id,) in matches:
        try:
            result = api.get_match_details(match_id=match_id)
        except (requests.exceptions.ConnectionError,  dota2api.src.exceptions.APIError, dota2api.src.exceptions.APITimeoutError):
            logging.warning("Api or timeout error in MatchDetails on matchid {}.".format(match_id))
            time.sleep(30)
            continue
        print("Added details for {}.".format(result["match_id"]))
        update_match_in_db(result)
        time.sleep(1)
    

def update_match_in_db(match):
#     print('Inside update_match_in_db')
    has_leavers = False
    for p in match["players"]:
        if p["leaver_status"] in [2,3,4,5,6]: 
            has_leavers = True
        connection = sqlite3.connect(database, timeout = 10)
        connection.execute("UPDATE OR IGNORE matches SET game_mode=?, has_leaver=?, duration=?, radiant_win=? WHERE match_id=?", (match["game_mode"], has_leavers, match["duration"], match["radiant_win"], match["match_id"]))
        connection.commit()
        connection.close()
        

match_details()

Inside do_match_details
Added details for 4022065818.
Added details for 4022065982.
Added details for 4022066022.
Added details for 4022066129.
Added details for 4022066206.
Added details for 4022066395.
Added details for 4022066547.
Added details for 4022066567.
Added details for 4022066592.
Added details for 4022066600.
Added details for 4022066689.
Added details for 4022066913.
Added details for 4022066925.
Added details for 4022067051.
Added details for 4022067086.
Added details for 4022067146.
Added details for 4022067218.
Added details for 4022067298.
Added details for 4022067616.
Added details for 4022067879.
Added details for 4022067930.
Added details for 4022067995.
Added details for 4022068331.
Added details for 4022068342.
Added details for 4022068644.
Added details for 4022068820.
Added details for 4022068883.
Added details for 4022069109.
Added details for 4022069182.
Added details for 4022069218.
Added details for 4022069347.
Added details for 4022069386.
Added details fo

In [5]:
match_history_thread = Thread(target=match_history)
match_history_thread.daemon = True

match_history_thread.start()

In [11]:
match_details_thread = Thread(target=match_details)
match_details_thread.daemon = True
match_details_thread.start()

Inside do_match_details


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\Madhukar\Anaconda2\Lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\Madhukar\Anaconda2\Lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-4-cd7012ba2fae>", line 5, in match_details
    do_match_details()
  File "<ipython-input-4-cd7012ba2fae>", line 18, in do_match_details
    result = api.get_match_details(match_id=match_id)
  File "C:\Users\Madhukar\Anaconda2\lib\site-packages\dota2api\__init__.py", line 122, in get_match_details
    return response.build(req, url, self.raw_mode)
  File "C:\Users\Madhukar\Anaconda2\lib\site-packages\dota2api\src\response.py", line 15, in build
    req_resp = req.json()
  File "C:\Users\Madhukar\Anaconda2\lib\site-packages\requests\models.py", line 826, in json
    return complexjson.loads(self.text, **kwargs)
  File "C:\Users\Madhukar\Anaconda2\Lib\json\__init__.py", line 339, in 